In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from keras.losses import MeanSquaredError

from keras.losses import mean_squared_error

# Load the models with custom objects
test_model = load_model('Models/test_model.h5', custom_objects={'mse': mean_squared_error})
valid_model = load_model('Models/valid_model.h5', custom_objects={'mse': mean_squared_error})

In [7]:
data = pd.read_csv('CSV_Files/glucose_data_resampled.csv')
data['Glucose_time'] = pd.to_datetime(data['Glucose_time'])
data.set_index('Glucose_time', inplace=True)

In [8]:
values = data['reading'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)

In [11]:
best_params_valid = pd.read_csv("Models/best_parameters_lstm.csv")
best_params_test = pd.read_csv("Models/best_parameters_test_lstm.csv")

# n_features,patience

n_feat_valid = best_params_valid['n_features'].values[0]
n_feat_test = best_params_test['n_features'].values[0]

patience_valid = best_params_valid['patience'].values[0]
patience_test = best_params_test['patience'].values[0]

In [13]:
input_data = scaled_values[-n_feat_valid:].reshape((1, n_feat_valid, 1))
predictions = []
for _ in range(10):
    next_value = valid_model.predict(input_data)
    predictions.append(next_value[0, 0])
    input_data = np.append(input_data[:, 1:, :], next_value).reshape((1, n_feat_valid, 1))

predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
[[109.04465 ]
 [108.33614 ]
 [107.496796]
 [106.25405 ]
 [104.63263 ]
 [102.7832  ]
 [100.87604 ]
 [ 99.07279 ]
 [ 97.413635]
 [ 95.87033 ]]
